In [13]:
import os, csv
import random, pydicom, numpy as np
import pandas as pd
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

In [14]:
#empty dictionary
pneumonia_locations = {}
with open(os.path.join('../input/stage_1_train_labels.csv'),
          mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader,None)
    for rows in reader:
        filename  = rows[0]
        location  = rows[1:5]
        pneumonia = rows[5]
        
        if pneumonia == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save pneumonia location in dictionary
            if filename in pneumonia_locations:
                pneumonia_locations[filename].append(location)
            else:
                pneumonia_locations[filename] = [location]
            
    

In [15]:
# load and shuffle filenames
folder = '../input/stage_1_train_images/'
filenames = os.listdir(folder)
random.shuffle(filenames)
# split into train and validation filenames
n_valid_samples = 2560 
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples





n train samples 23124
n valid samples 2560


### Data Generator

In [16]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=320, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in pneumonia_locations:
            # loop through pneumonia
            for location in pneumonia_locations[filename]:
                # add 1's at the location of the pneumonia
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        
        img = img - np.ndarray.mean(img)
        
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [17]:
''' MINE
class generator(keras.utils.Sequence):

    def __init__(self, folder, filenames, pneumonia_locations
                =None, batch_size=32, image_size=320, 
                shuffle=True,augment=False,predict=False):
        self.folder = folder
        self.filenames = filenames 
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
    
    def __load__(self,filename):
        img = pydicom.dcmread(os.path.join(self.folder,
                                           filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in pneumonia_locations:
            # loop through penumonia 
            for location in pneumonia_locations[filename]:
                # add 1's in the location of the pneumonia
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
            if self.augment and random.random()>0.5:
                img = np.fliplr(img)
                msk = np.fliplr(msk)
            # resize both image and mask
            img = resize(img, (self.image_size, 
                               self.image_size), 
                               mode='reflect')
            msk = resize(msk, (self.image_size, 
                               self.image_size), 
                               mode='reflect') > 0.5
            # add training channel dimension
            img = np.expand_dims(img,-1)
            msk = np.expand_dims(msk,-1)
            return img, msk
        
    def __loadpredict__(self,filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder,
                              filename)).pixel_array
        # resize image
        img = resize(img,(self.image_size, 
                          self.image_size),mode='reflect')
        
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
    
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size
                    :(index+1)*self.batch_size]
        #predict mode: return images and filenames
        if self.predict:
            # load files; why do we do this ? 
            imgs = [self.__loadpredict__(filename) for 
                    filename in filenames]
            # create numpy batch; why do we do this ? 
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else: 
            # load files
            items = [self.__load__(filename) for filename
                    in filenames]
            # unzip images and masks
            imgs,msks = zip(*items)
            
            imgs,msks = items
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
            
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames)/
                               self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames)/
                       self.batch_size)
'''

" MINE\nclass generator(keras.utils.Sequence):\n\n    def __init__(self, folder, filenames, pneumonia_locations\n                =None, batch_size=32, image_size=320, \n                shuffle=True,augment=False,predict=False):\n        self.folder = folder\n        self.filenames = filenames \n        self.pneumonia_locations = pneumonia_locations\n        self.batch_size = batch_size\n        self.image_size = image_size\n        self.shuffle = shuffle\n        self.augment = augment\n        self.predict = predict\n        self.on_epoch_end()\n    \n    def __load__(self,filename):\n        img = pydicom.dcmread(os.path.join(self.folder,\n                                           filename)).pixel_array\n        # create empty mask\n        msk = np.zeros(img.shape)\n        # get filename without extension\n        filename = filename.split('.')[0]\n        # if image contains pneumonia\n        if filename in pneumonia_locations:\n            # loop through penumonia \n           

### Newtork 

In [18]:
BATCH_SIZE = 16
IMAGE_SIZE = 320

In [19]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(256, 1, activation=None)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2DTranspose(128, (8,8), (4,4), padding="same", activation=None)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**(depth-2))(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [20]:
'''
mine
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same',
                           use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    
def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(
    inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same'
                            ,use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same'
                           ,use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size,channels,n_blocks=2,
                   depth=4):
    # input:
    inputs = keras.Input(shape=(input_size, input_size,
                                1))
    x = keras.layers.Conv2D(channels,3,padding='same',
                            use_bias=False)(inputs)
    # residual blocks; 
    # why do we do this ? 
    
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
        # output
        x = keras.layers.BatchNormalization(momentum=0.9)(x)
        x = keras.layers.LeakyReLU(0)(x)
        x = keras.layers.Conv2D(256,1,activation=None)(x)
        x = keras.layers.BatchNormalization(momentum=0.9)(x)
        x = keras.layers.Conv2DTranspose(128,(8,8),(4,4),
        padding="same",activation=None)(x)
        x = keras.layers.BatchNormalization(momentum=0.9)(x)
        x = keras.layers.LeakyReLU(0)(x)
        x = keras.layers.Conv2D(1,1, activation='sigmoid')(
        x)
        outputs = keras.layers.UpSampling2D(2**(depth-2))(x)
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
'''

'\nmine\ndef create_downsample(channels, inputs):\n    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)\n    x = keras.layers.LeakyReLU(0)(x)\n    x = keras.layers.Conv2D(channels, 1, padding=\'same\',\n                           use_bias=False)(x)\n    x = keras.layers.MaxPool2D(2)(x)\n    \ndef create_resblock(channels, inputs):\n    x = keras.layers.BatchNormalization(momentum=0.9)(\n    inputs)\n    x = keras.layers.LeakyReLU(0)(x)\n    x = keras.layers.Conv2D(channels, 3, padding=\'same\'\n                            ,use_bias=False)(x)\n    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)\n    x = keras.layers.LeakyReLU(0)(x)\n    x = keras.layers.Conv2D(channels, 3, padding=\'same\'\n                           ,use_bias=False)(x)\n    return keras.layers.add([x, inputs])\n\ndef create_network(input_size,channels,n_blocks=2,\n                   depth=4):\n    # input:\n    inputs = keras.Input(shape=(input_size, input_size,\n                                1))\n

In [21]:
#Train Network

In [22]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=IMAGE_SIZE, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 20
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

# create train and validation generators
folder = '../input/stage_1_train_images'
train_gen = generator(folder, train_filenames, pneumonia_locations, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, shuffle=True, augment=True, predict=False)
valid_gen = generator(folder, valid_filenames, pneumonia_locations, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, shuffle=False, predict=False)

print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 320, 320, 32) 288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_24 (BatchNo (None, 320, 320, 32) 128         conv2d_24[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_24 (LeakyReLU)      (None, 320, 320, 32) 0           batch_normalization_24[0][0]     
__________________________________________________________________________________________________
conv2d_25 

In [23]:
'''
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true,[-1])
    y_pred = tf.reshape(y_pred,[-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.)/ (tf.reduce_sum(y_true)
           + tf.reduce_sum(y_pred)-intersection + 1.)
    return 1-score

def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true
    , y_pred) + 0.5 * iou_loss(y_true, y_pred)

def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true* y_pred, axis=[1,
                2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce.mean((intersect + smooth)/ (union-
        -intersect + smooth))

# create network and compiler
model = create_network(input_size=IMAGE_SIZE,channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',loss=iou_bce_loss, 
             metrics=['accuracy',mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 20
    return lr*(np.cos(np.pi*x/epochs)+1.)/2

learning_rate = tf.keras.callbacks.LearningRateScheduler
(cosine_annealing)
folder = '../input/stage_1_train_images/'
train_gen = generator(folder, train_filenames, pneumonia_locations, 
            batch_size=BATCH_SIZE,
            image_size=IMAGE_SIZE, shuffle=True,
            augment=True, predict=False)
valid_gen = generator(folder, valid_filenames, pneumonia_locations, 
            batch_size=BATCH_SIZE,
            image_size=IMAGE_SIZE, shuffle=True,
            augment=True, predict=False)

print(model.summary())    
'''    

"\ndef iou_loss(y_true, y_pred):\n    y_true = tf.reshape(y_true,[-1])\n    y_pred = tf.reshape(y_pred,[-1])\n    intersection = tf.reduce_sum(y_true * y_pred)\n    score = (intersection + 1.)/ (tf.reduce_sum(y_true)\n           + tf.reduce_sum(y_pred)-intersection + 1.)\n    return 1-score\n\ndef iou_bce_loss(y_true, y_pred):\n    return 0.5 * keras.losses.binary_crossentropy(y_true\n    , y_pred) + 0.5 * iou_loss(y_true, y_pred)\n\ndef mean_iou(y_true, y_pred):\n    y_pred = tf.round(y_pred)\n    intersect = tf.reduce_sum(y_true* y_pred, axis=[1,\n                2,3])\n    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])\n    smooth = tf.ones(tf.shape(intersect))\n    return tf.reduce.mean((intersect + smooth)/ (union-\n        -intersect + smooth))\n\n# create network and compiler\nmodel = create_network(input_size=IMAGE_SIZE,channels=32, n_blocks=2, depth=4)\nmodel.compile(optimizer='adam',loss=iou_bce_loss, \n             metrics=['accuracy',mean_

In [ ]:
history = model.fit_generator(train_gen,validation_data
=valid_gen, callbacks=[learning_rate], epochs=20, shuffle=True)

Epoch 1/20
1445/1445 [==============================] - 401s 278ms/step - loss: 0.4800 - acc: 0.9614 - mean_iou: 0.5662 - val_loss: 0.4654 - val_acc: 0.9668 - val_mean_iou: 0.6253
Epoch 2/20
1445/1445 [==============================] - 400s 277ms/step - loss: 0.4591 - acc: 0.9638 - mean_iou: 0.6040 - val_loss: 0.4491 - val_acc: 0.9583 - val_mean_iou: 0.5911
Epoch 3/20
1445/1445 [==============================] - 400s 277ms/step - loss: 0.4431 - acc: 0.9670 - mean_iou: 0.6445 - val_loss: 0.4370 - val_acc: 0.9630 - val_mean_iou: 0.5862
Epoch 4/20
1445/1445 [==============================] - 400s 277ms/step - loss: 0.4323 - acc: 0.9686 - mean_iou: 0.6570 - val_loss: 0.4389 - val_acc: 0.9572 - val_mean_iou: 0.5823
Epoch 5/20
1445/1445 [==============================] - 400s 277ms/step - loss: 0.4269 - acc: 0.9691 - mean_iou: 0.6602 - val_loss: 0.4226 - val_acc: 0.9680 - val_mean_iou: 0.6406
Epoch 6/20
1445/1445 [==============================] - 400s 277ms/step - loss: 0.4200 - acc: 0.9700

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(131)
plt.plot(history.epoch, history.history["loss"], label="Train loss")
plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(history.epoch, history.history["acc"], label="Train accuracy")
plt.plot(history.epoch, history.history["val_acc"], label="Valid accuracy")
plt.legend()
plt.subplot(133)
plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.show()

In [ ]:
# load and shuffle filenames
folder = '../input/stage_1_test_images'
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=16, image_size=IMAGE_SIZE, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break
        
print("Done predicting...")
        
# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('submission.csv')